# Day3 任意課題

## 宿題の内容



1.   独自の質問と参照資料の作成
  *   自分で5つ以上の質問文を考案してください
  *    各質問に対する回答を含む参照文書を用意してください
  *   少なくとも1つは、LLMが単体では正確に答えられないような知識を含む質問にしてください
2.   実験の実施
  * 演習で使用したコードをベースに、以下の2つの方法で回答を生成してください
      * ベースのLLM(RAGなし)での回答生成
      * RAGを組み合わせた回答生成
  * 回答の評価では、単純なYes/No判定でも良いです
      * より詳細な評価指標も検討していただけるとなお良いです
3. 結果分析と考察
  * 生成した結果をまとめ、RAGありとRAGなしの差異を分析してください
  * RAGによって回答が改善したケースと悪化したケースの両方について考察してください
  * 結果に基づいて、RAGの有効性と限界についての考察を記述してください

以下をHomework.mdにまとめる。

* 質問設計の観点と意図
* RAGの実装方法と工夫点
* 結果の分析と考察
* 発展的な改善案(任意)


## 初期設定

In [1]:
!pip install --upgrade transformers
!pip install google-colab-selenium
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
# 演習用のコンテンツを取得
!git clone https://github.com/matsuolab/lecture-ai-engineering.git

Cloning into 'lecture-ai-engineering'...
remote: Enumerating objects: 81, done.
remote: Total 81 (delta 0), reused 0 (delta 0), pack-reused 81 (from 2)
Receiving objects: 100% (81/81), 1.34 MiB | 21.10 MiB/s, done.
Resolving deltas: 100% (12/12), done.


In [7]:
# HuggingFace Login
from huggingface_hub import notebook_login

notebook_login()

In [4]:
# CUDAが利用可能ならGPUを、それ以外ならCPUをデバイスとして設定
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
import random
random.seed(0)

In [8]:
# モデル(Gemma2)の読み込み

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "google/gemma-2-2b-jpn-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
        )

tokenizer_config.json:   0%|          | 0.00/46.9k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [9]:
def generate_output(query):
  messages = [
      {"role": "user", "content": query},
  ]
  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      max_new_tokens=256,
      eos_token_id=terminators,
      do_sample=False,
      # temperature=0.6, # If do_sample=True
      # top_p=0.9,  # If do_sample=True
  )

  response = outputs[0][input_ids.shape[-1]:]
  return tokenizer.decode(response, skip_special_tokens=True)

## 質問文を考える

AIが事前知識を持たないという点では、最新のリーク情報やニュースを求めるプロンプトをいれると性能評価につながりそう。

###1つ目の質問


*   「Material 3 Expressiveについて教えてください。」
*   比較的直近の情報を求めており、RAGなしではハルシネーションを起こす可能性が非常に高い

模範解答

```
Material 3 Expressiveとは、Googleが開発した、新しいUXデザインシステムです。Material 3 Expressiveは、大胆な形と色使いが特徴で、快適なユーザーエクスペリエンスを生み出します。開発元のGoogleは、Material 3 Expressiveは、「大胆なデザインの新たな方向性」だと提唱しています。
```




**利用できる情報（RAGあり）**

> Googleは14日(日本時間)に開発者向け会議「Google I/O」を開催した。イベントではAndroid 16およびWear OS 6において、「Material 3 Expressive」と呼ばれるデザインの刷新が取り入れられると発表した。
> Material 3 Expressiveは数年ぶりの大型アップデートとなっており、個性を表現する新たな方法を提供、滑らかな操作感と、ひと目で分かる便利な情報表示を実現したという。

>「Material 3 Expressive」はGoogleが提唱する「大胆なデザインの新たな方向性」で、Googleは「Googleのデザインシステムにおける、これまでで最も綿密な調査に基づいたアップデート」とうたっています。Material 3 Expressiveを作成したGoogleのマテリアルデザインチームは、「なぜアプリはどれも似たような見た目になっているのだろう？退屈なのだろう？もっと雰囲気を変える余地はなかったのだろうか？」という疑問を抱き、新しいUXデザインシステムとしてMaterial 3 Expressiveを作成しました。

> Material 3 Expressiveの開発に向け、Googleはユーザーがどこに注目したかを分析する「視線追跡」、さまざまなデザインに対する感情的な反応を測定する「アンケート調査およびフォーカスグループ」、ユーザーの感情と好みを分析する「実験」、参加者がインターフェースをどれだけ早く理解し、使用できるかを確認する「ユーザビリティ」という4種類の研究を実施しました。

>GoogleはMaterial 3 Expressiveについて、「表現力豊かなデザインの基本は、色・形・サイズ・動き・空間の表現です。Material 3 Expressiveは、大胆な形と色使いが特徴で、快適なユーザーエクスペリエンスを生み出します」と説明しています。

> Material Youをベースに多くのカスタマイズをオプションを提供し、自分らしさのより豊かな表現を実現。より自然で弾むようなアニメーションシステムを導入し、日常の操作にささやかな喜びを添えるという

> 進化したカラーテーマ、画面サイズに合わせたコンポーネントの自動調整、強調されたタイポグラフィにより、見た目や雰囲気をカスタマイズ可能。Googleアプリ全体で適用される

> Wear OSでは円形ディスプレイを生かした流れるような操作感を提供。アニメーションがディスプレイ2曲線に沿って動き、リストをスクロールすると奥行き感が生まれるなど、情報の視認性が向上。また、動的カラーテーマにより、選択した文字盤のテーマがシステム全体に適用。性能向上と効率最適化により、バッテリ寿命が最大10%向上

> これらのデザイン改善はAndroid 16/Wear OS 6端末向けにパートナー企業と共同開発しており、2025年後半にGoogle Pixel端末で先行体験できるという。

> このほか、AIアシスタントの「Gemini」がWear OSにも搭載されることを発表。また、Android AutoおよびGoogle搭載自動車でも、Geminiにより音声コマンド体験が改善される。

>合わせて、Gemini Liveのカメラおよび画面共有機能は、Gemini Advancedサブスクリプションなしで全Androidユーザーが利用可能になったことも発表された。

>さらに、「デバイスを探す」機能が「Find Hub」に進化。端末や持ち物の位置確認、大切な人の帰宅確認、夜のお出かけ中の位置情報共有などを一元的に管理できる。2025年後半以降は、衛星接続に対応した端末/キャリアを対象に、Find Hubの衛星接続機能への対応を予定しており、携帯電話の電波が届かない場所でも友人や家族と連絡可能としている。



引用：*https://pc.watch.impress.co.jp/docs/news/2013825.html*

引用：*http://gigazine.net/news/20250506-android-material-3-expressive/*



In [10]:
question =  "Material 3 Expressiveについて教えてください"
response = generate_output(question)
print(response)

## Material 3 Expressive について

**Material 3 Expressive** は、**Adobe** が開発した、**3D モデルの表現力向上**を目的とした、**新しい技術**です。

**主な特徴:**

* **リアルな質感と光:**  3D モデルの質感と光をよりリアルに表現します。
* **複雑な形状の表現:**  複雑な形状や多面的なオブジェクトを表現できます。
* **視覚的な魅力:**  より魅力的で、ユーザーに訴求力のある3Dモデルを作成できます。
* **柔軟性:**  様々な素材や質感、光を組み合わせることで、多様な表現が可能になります。

**具体的な活用例:**

* **ゲーム:**  よりリアルなキャラクターや環境を表現
* **アニメーション:**  より魅力的なキャラクターやシーンを表現
* **VR/AR:**  より没入感のある体験を提供
* **製品デザイン:**  製品の質感や形状をより正確に表現


**メリット:**

* **よりリアルな3Dモデル:**  従来の技術では難しい複雑な形状や質感の表現が可能
* **多様な表現:**  素材や質感、光を自由に組み合わせることで、


Material 3 ExpressiveはAdobeではなくGoogleが開発したものであり、目的は3Dモデルの表現力向上を目的としたものではない。残念。

Geminiを利用して、スコアを求めていく。

In [37]:
# @title 評価実装
gold_answer = "Material 3 Expressiveとは、Googleが開発した、新しいUXデザインシステムです。Material 3 Expressiveは、大胆な形と色使いが特徴で、快適なユーザーエクスペリエンスを生み出します。開発元のGoogleは、Material 3 Expressiveは、「大胆なデザインの新たな方向性」だと提唱しています。"

import google.generativeai as genai
from google.colab import userdata

# Gemini 2.0 Flash モデルを指定
model = genai.GenerativeModel('gemini-2.0-flash')
genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))

def gemini_generator(query):
    response = model.generate_content(query)
    return response.text

def evaluate_answer_accuracy(query, response, reference):

    template_accuracy1 = (
        "Instruction: You are a world-class state-of-the-art assistant for rating "
        "a User Answer given a Question. The Question is completely answered by the Reference Answer.\n"
        "Say 4, if User Answer is fully contained and equivalent to Reference Answer "
        "in all terms, topics, numbers, metrics, dates, and units.\n"
        "Say 2, if User Answer is partially contained and almost equivalent to Reference Answer "
        "in all terms, topics, numbers, metrics, dates, and units.\n"
        "Say 0, if User Answer is not contained in Reference Answer or not accurate in all terms, topics, "
        "numbers, metrics, dates, and units, or the User Answer does not answer the question.\n"
        "Do not explain or justify your rating. Your rating must be only 4, 2, or 0 according to the instructions above.\n"
        "Even small discrepancies in meaning, terminology, directionality, or implication must result in a lower score. Only rate 4 if the User Answer is a complete and precise match to the Reference Answer in every aspect.\n"
        "### Question: {query}\n"
        "### {answer0}: {sentence_inference}\n"
        "### {answer1}: {sentence_true}\n"
        "The rating is:\n"
    )
    template_accuracy2 = (
        "I will rate the User Answer in comparison to the Reference Answer for a given Question.\n"
        "A rating of 4 indicates that the User Answer is entirely consistent with the Reference Answer, covering all aspects, topics, numbers, metrics, dates, and units.\n"
        "A rating of 2 signifies that the User Answer is mostly aligned with the Reference Answer, with minor discrepancies in some areas.\n"
        "A rating of 0 means that the User Answer is either inaccurate, incomplete, or unrelated to the Reference Answer, or it fails to address the Question.\n"
        "I will provide the rating without any explanation or justification, adhering to the following scale: 0 (no match), 2 (partial match), 4 (exact match).\n"
        "Even minor inconsistencies in meaning, terminology, emphasis, or factual detail should prevent a rating of 4. Only assign a 4 if the User Answer exactly and unambiguously matches the Reference Answer in every respect.\n"
        "Do not explain or justify my rating. My rating must be only 4, 2, or 0 only.\n\n"
        "Question: {query}\n\n"
        "{answer0}: {sentence_inference}\n\n"
        "{answer1}: {sentence_true}\n\n"
        "Rating: "
    )

    score1_response = gemini_generator(
        template_accuracy1.format(
            query=query,
            answer0="User Answer",
            answer1="Reference Answer",
            sentence_inference=response,
            sentence_true=reference,
        )
    )
    try:
        score1 = int(score1_response)
    except ValueError:
        print(f"Failed to parse score1: {score1_response}")
        score1 = 0

    score2_response = gemini_generator(
        template_accuracy2.format(
            query=query,
            answer0="Reference Answer",
            answer1="User Answer",
            sentence_inference=reference,
            sentence_true=response,
        )
    )
    try:
        score2 = int(score2_response)
    except ValueError:
        print(f"Failed to parse score2: {score2_response}")
        score2 = 0

    return (score1 + score2) / 2

In [15]:
score = evaluate_answer_accuracy(question, response, gold_answer)
print(score)

0.0


### 2つ目の質問





## RAGを実装する



### 1つ目の質問

In [16]:
from sentence_transformers import SentenceTransformer

emb_model = SentenceTransformer("infly/inf-retriever-v1-1.5b", trust_remote_code=True)
# In case you want to reduce the maximum length:
emb_model.max_seq_length = 4096

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/284 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/19.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/55.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/918 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [17]:
with open("/content/lecture-ai-engineering/day3/data/Day3_Material3_RAG.txt", "r") as f:
  raw_writedown = f.read()

In [21]:
# ドキュメントを用意する。
documents = [text.strip() for text in raw_writedown.split("。")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[1])

ドキュメントサイズ:  21
ドキュメントの例: 
 イベントではAndroid 16およびWear OS 6において、「Material 3 Expressive」と呼ばれるデザインの刷新が取り入れられると発表した


In [34]:
# Retrievalの実行
question = "Material 3 Expressiveについて教えてください"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

[[53.62813949584961, 70.31429290771484, 75.94002532958984, 71.2515869140625, 73.00675201416016, 70.56040954589844, 74.82502746582031, 78.21403503417969, 68.39616394042969, 58.0240364074707, 55.09220504760742, 57.42178726196289, 55.09043502807617, 54.52939224243164, 55.68000793457031, 54.308109283447266, 56.14018249511719, 56.24114990234375, 55.62900161743164, 53.96223068237305, 49.44849395751953]]


In [35]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

取得したドキュメント1: (Score: 78.21403503417969)
Material 3 Expressiveは、大胆な形と色使いが特徴で、快適なユーザーエクスペリエンスを生み出します」と説明しています 


取得したドキュメント2: (Score: 75.94002532958984)
Material 3 Expressiveは数年ぶりの大型アップデートとなっており、個性を表現する新たな方法を提供、滑らかな操作感と、ひと目で分かる便利な情報表示を実現したという 


取得したドキュメント3: (Score: 74.82502746582031)
GoogleはMaterial 3 Expressiveについて、「表現力豊かなデザインの基本は、色・形・サイズ・動き・空間の表現です 


取得したドキュメント4: (Score: 73.00675201416016)
Material 3 Expressiveを作成したGoogleのマテリアルデザインチームは、「なぜアプリはどれも似たような見た目になっているのだろう？退屈なのだろう？もっと雰囲気を変える余地はなかったのだろうか？」という疑問を抱き、新しいUXデザインシステムとしてMaterial 3 Expressiveを作成しました 


取得したドキュメント5: (Score: 71.2515869140625)
「Material 3 Expressive」はGoogleが提唱する「大胆なデザインの新たな方向性」で、Googleは「Googleのデザインシステムにおける、これまでで最も綿密な調査に基づいたアップデート」とうたっています 




In [36]:
references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
query =  f"[参考資料]\n{references}\n\n[質問] Material 3 Expressiveについて教えてください"
response = generate_output(query)
print(response)

## Material 3 Expressive について

**Material 3 Expressive** は、Googleが提唱する新しいデザインシステムです。大胆な形と色使いが特徴で、ユーザーエクスペリエンスを向上させることを目指しています。

**主な特徴は以下の通りです:**

* **個性を表現する新たな方法:**  従来のMaterialデザインとは異なり、より自由なデザイン表現が可能になりました。
* **滑らかな操作感と便利な情報表示:**  操作のしやすさと、ユーザーがすぐに理解できる情報表示を実現しています。
* **大胆なデザインの新たな方向性:**  Googleが提唱する「Googleのデザインシステムにおける、これまでで最も綿密な調査に基づいたアップデート」です。
* **色・形・サイズ・動き・空間の表現:**  デザインの基本要素を多角的に表現することで、より魅力的なユーザーエクスペリエンスを実現しています。


**Material 3 Expressive の開発背景:**

* Googleは、アプリの見た目が似たようなものになりがちだと感じ、ユーザーエクスペリエンスの飽きを感じていました。
*  新しいUXデザインシステムとしてMaterial 3 Expressiveを作成しました。



 





In [38]:
score = evaluate_answer_accuracy(question, response, gold_answer)
print(score)

2.0


ニュース記事などの比較的整理された情報を事前に渡しておくことで、かなり精度の高い回答を生成することができた。